# **DATA SCIENCE PARA MATCH DE CURRÍCULOS COM VAGAS DE EMPREGO**

---
## **Importação dos CVs**

In [ ]:
#instalando pacotes necessários
!pip install pdfplumber
import pdfplumber
import nltk  
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
#ler o curriculo
arquivoPDF = pdfplumber.open('Igor.pdf')

In [ ]:
#apresentar os metadados do currículo
arquivoPDF.metadata

In [ ]:
#apresentar os hyperlinks existentes no currículo
arquivoPDF.hyperlinks

In [ ]:
print(textoCRU)

---
## **Pré-processamento de Texto**

In [ ]:
# A função word_tokenize() quebra o texto em palavras individuais.
nltk.download('punkt')
lista_de_palavras = nltk.tokenize.word_tokenize(textoCRU)
lista_de_palavras

In [ ]:
# Padronizando as palavras em lowercase (apenas letras minúsculas)
lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]
lista_de_palavras

In [ ]:
#Criando uma lista que contém pontuação que desejamos remover
pontuacao = ['(',')',';',':','[',']',',']

In [ ]:
#Criando uma lista de stop words "a", "de", "um"que não tem valor como palavra
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('portuguese')
stop_words

In [ ]:
#criando uma lista de palavra sem stopword e pontuações
keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuacao]
keywords

In [ ]:
#numero de palavras das reviews
len(keywords)

In [ ]:
# concatenar as palavras
textocv = " ".join(s for s in keywords)
textocv

---
## **WordCloud - Nuvem de Palavras**

In [ ]:
# gerar uma wordcloud
from wordcloud import WordCloud

wordcloud = WordCloud(background_color = '#0f54c9', 
                      max_font_size = 150, 
                      width = 1280, 
                      height = 720, 
                      colormap= 'Blues').generate(textocv) 

#mostrar a imagem final
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(16, 9))
ax.imshow(wordcloud)
ax.set_axis_off()
plt.imshow(wordcloud)
wordcloud.to_file("wordcloud.png")
plt.show()

In [ ]:

from wordcloud import WordCloud

#funcao para gerar a wordcloud
def NuvemDePalavras(cv, salvar = True):
    '''
    cv: caminho de um arquivo PDF 
    '''
    arquivoPDF = pdfplumber.open(cv)
    primeira_pagina = arquivoPDF.pages[0]  #lê apenas a primeira página
    textoCRU = primeira_pagina.extract_text()

    lista_de_palavras = nltk.tokenize.word_tokenize(textoCRU)  # transforma o texto cru em uma lista de termos
    lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]  # deixando tudo minusculo

    keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuacao]  # tira as pontuacoes e stopwords 
    textocv = " ".join(s for s in keywords)  # junta tudo em um texto só novamente. 


    wordcloud = WordCloud(background_color = '#0f54c9', 
                          max_font_size = 150, 
                          width = 1280, 
                          height = 720, 
                          colormap= 'Blues').generate(textocv) 
 
    # mostrar a imagem final
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.imshow(wordcloud)
    ax.set_axis_off()
    plt.imshow(wordcloud)

    if salvar:
        wordcloud.to_file("wordcloud.png")

    plt.show()


In [ ]:
NuvemDePalavras('Igor.pdf')

---
## **Entrada das Vagas de Emprego**

- **Vaga 1 - DS Senior:** https://www.linkedin.com/jobs/view/2592718801/?alternateChannel=search&refId=HcViZqV2vjZAXA0OvOZrjQ%3D%3D&trackingId=6n427gmxJ%2BH5bnRHI0T5Lg%3D%3D
- **Vaga 2 - DS Junior:** https://www.linkedin.com/jobs/view/2621616275/?alternateChannel=search&refId=I4M3V3mzbgz5oU5EfgwJxw%3D%3D&trackingId=4gDNWcSbeeVhAr15Vs2kaA%3D%3D
- **Vaga 3 - Engenheiro de Dados:** https://www.linkedin.com/jobs/view/2596248068/?alternateChannel=search&refId=sI8VCh%2FcadPug6NsBop4sA%3D%3D&trackingId=uMjFL1VlttYcK4XiXkcToA%3D%3D 
- **Vaga 4 - Marketing:**  https://www.linkedin.com/jobs/view/2601831935/?alternateChannel=search&refId=J6jCHp9UlXvGcv2qAD132w%3D%3D&trackingId=cZ4J1BVrDvBhutTRklbPPw%3D%3D

In [ ]:
#ler as palavras chaves do emprego e seus respectivos pesos
import pandas as pd

vagas = pd.read_excel('vagas.xlsx', sheet_name = None)

In [ ]:
n_vagas = len(vagas.keys())
nome_vagas = list(vagas.keys())

In [ ]:
# transformando a saida numa lista de dataframes (cada dataframe representa uma vaga)
vagas = [vagas[nome_vagas[i]] for i in range(n_vagas)]

In [ ]:
vagas[0]

In [ ]:
vaga1 = vagas[0]
vaga1

In [ ]:
palavras_chaves = list(vaga1['palavras-chave'])
palavras_chaves

In [ ]:
# Clear Sale Data Science Analytics Senior
import numpy as np

limite = 5
  
pesos = list(vaga1['pesos'])
palavras_chaves = list(vaga1['palavras-chave'])
pmax = np.sum(np.array(pesos) * limite)
print(pmax) 

In [ ]:
# contando as palavras chaves que ocorrem no CV

cont = [textocv.count(pc) for pc in palavras_chaves]
cont

In [ ]:
# limitar pelo threshold
def aux(x):
    return x if x <= limite else limite

cont = [aux(i) for i in cont]
cont

In [ ]:
#score do currículo com a vaga
score = ((np.array(cont) * pesos).sum()/pmax).round(4)
score

---
## **Match dos CVs com as Vagas**

In [ ]:
# função entrada: CV saida, vaga - saida: score

pontuacao = ['(', ')', ';', ':', '[', ']', ',']
stop_words = nltk.corpus.stopwords.words('portuguese')

def MatchCV(cv, vaga, limite = 5):
    '''
    cv: caminho de um arquivo PDF
    vaga: dataset de palavras-chave e pesos
    '''

    arquivoPDF = pdfplumber.open(cv)
    primeira_pagina = arquivoPDF.pages[0]  #lê apenas a primeira página
    textoCRU = primeira_pagina.extract_text()

    lista_de_palavras = nltk.tokenize.word_tokenize(textoCRU)  # transforma o texto cru em uma lista de termos
    lista_de_palavras = [palavra.lower() for palavra in lista_de_palavras]  # deixando tudo minusculo

    keywords = [palavra for palavra in lista_de_palavras if not palavra in stop_words and not palavra in pontuacao]  # tira as pontuacoes e stopwords 
    textocv = " ".join(s for s in keywords)  # junta tudo em um texto só novamente. 

    pesos = list(vaga['pesos'])
    palavras_chaves = list(vaga['palavras-chave'])

    cont = [textocv.count(pc) for pc in palavras_chaves]  # conta quantas vezes cada termo da vaga aparece no texto do cv

    def aux(x, limite):
        return x if x <= limite else limite

    cont = [aux(i, limite) for i in cont]   # coloca o limite na contagem de palavras

    pmax = np.sum(np.array(pesos) * limite) 

    score = ((np.array(cont) * pesos).sum()/pmax).round(4)

    return score

In [ ]:
MatchCV('Igor.pdf', vagas[2])

In [ ]:
lista_de_vagas = vagas

lista_de_cvs = ['Igor.pdf']


In [ ]:
lista_de_vagas[0]

In [ ]:
# Lista de Listas, cada lista interna é score de uma pessoa nas vagas
pessoas = [[MatchCV(cv, vaga) for vaga in lista_de_vagas] for cv in lista_de_cvs]
pessoas

In [ ]:
# filtro para pegar apenas os nomes das pessoas na lista de caminhos
nomes = [cv.split('/')[-1].split('.')[0] for cv in lista_de_cvs]

In [ ]:
# Conjunto de dados comos matches das pessoas com as vagas
matchs = pd.DataFrame(pessoas, columns = nome_vagas, index = nomes)

In [ ]:
matchs.sort_values(by = 'eng_dados', ascending = False)